In [1]:
"""
Предобработка данных ( талица пост обработана заранее нейросетью) и обучение модели
"""

'\nПредобработка данных ( талица пост обработана заранее нейросетью) и обучение модели\n'

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np

from category_encoders.count import CountEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import string
from sklearn.decomposition import PCA
np.random.seed(74)
from catboost import CatBoostClassifier


In [3]:
def get_score(y,X,y_tr, X_tr, model):
    """
    Получает на выход данные терйна и теста, а также обученыю модель.
    Функция возвращает Precision, Recall, f1, ROC-AUC на трейне и на тесте.
    А так же кол-во положительных передположений (лайков) на трейне и тесте.
    """
    from sklearn.metrics import log_loss, roc_auc_score
    
    from sklearn.metrics import precision_score, recall_score, f1_score
    precision_train = precision_score(y_tr, model.predict(X_tr),  average='macro')
    recall_train = recall_score(y_tr, model.predict(X_tr),  average='macro')
    f1_train= f1_score(y_tr, model.predict(X_tr),  average='macro')
    roc_train = roc_auc_score(y_tr, model.predict(X_tr),  average='macro')
    
    precision = precision_score(y, model.predict(X),  average='macro')
    recall = recall_score(y, model.predict(X),  average='macro')
    f1= f1_score(y, model.predict(X),  average='macro')
    roc = roc_auc_score(y, model.predict(X), average='macro')
    
    result_1_train = sum(model.predict(X_tr))
    result_1_test = sum(model.predict(X))
    
    return [[f'precision_tr= {precision_train:.4f}, recall_tr= {recall_train:.4f}, f1_tr= {f1_train:.4f}, ROC-AUC_tr= {roc_train:.4f}'],
            [f'precision= {precision:.4f}, recall= {recall:.4f}, f1= {f1:.4f}, ROC-AUC= {roc:.4f}'], 
            [f'На трейне предположили {result_1_train}, на тесте предположили {result_1_test}']]


In [4]:
def transform_feed(feed):
    #предобрабатывает таблицу feed
    feed_ = feed.copy()
    feed_ = feed_.drop_duplicates()
    return feed_

In [5]:
def transform_post(post, n_pca = 20, drop = True, ohe = True,lemma =True):
    """
    Функция получает на вход таблицу пост post.
    Опционально проводит OneHotEncoding типов постов,
    Лемматизацию текстов, Tf-Idf и накладывает на полученые вектора  PCA
    
    :n_pca: кол-во изерений в новом пространстве, если 0 не проводит pca b tf-idf
    :drop: = True/False дропать ли колонку текст
    :ohe: = True/False проводить ли  OneHotEncoding колонки topic
    :lemma: =True/False проводить ли лемматизацию
       
    """
    post_ = post.copy()
    wnl = WordNetLemmatizer()
    
    if lemma == True:
        def preprocessing(line, token=wnl):
            line = line.lower()
            line = re.sub(r"[{}]".format(string.punctuation), " ", line)
            line = line.replace('\n\n', ' ').replace('\n', ' ')
            line = ' '.join([token.lemmatize(x) for x in line.split(' ')])
            return line
    else:
        preprocessing = None

    #учу OHE_Post
    if ohe == True:
        one_hot = pd.get_dummies(post_['topic'], prefix='topic', drop_first=True)
        post_  = pd.concat((post_.drop(['topic'], axis=1), one_hot), axis=1)
        post_transformd = post_
    elif ohe == False:
        post_transformd = post_.drop(['topic'],axis =1 )
    
    if n_pca > 0 :
        #провожу tf-idf для Post
    
        tf = TfidfVectorizer(stop_words='english',
                             preprocessor=preprocessing, 
                             min_df = 5) #создаю экземпляр класса
        tf_idf_ = tf.fit_transform(post_['text'])#учу класс
        tf_idf_ = tf_idf_.toarray() - tf_idf_.mean() #центрируем данные

        list_col_pca = [f"PCA_{nn}" for nn in range(1,n_pca + 1)] 
        #ПРовожу PCA для User
        pca = PCA(n_components=n_pca,random_state = 74)
        #создаю экземплря PCA
        PCA_dataset = pca.fit_transform(tf_idf_) #провожу PCA 
        PCA_dataset = pd.DataFrame(PCA_dataset, columns=list_col_pca,index=post.index)
        #Трансформирую Post
    
        post_transformd = pd.concat((post_transformd, PCA_dataset), axis=1)
        if drop == True:
            post_transformd = post_transformd.drop(['text'],axis=1) 
            
    
    else:
        if drop == True:
            post_transformd = post_transformd.drop(['text'],axis=1)
        
    return post_transformd

In [6]:
def transform_user_with_drop_fich(user,drop_col = [],ohe_col=[], count_city = True, count_country = True, age_group = 'group', norm = True):
    """
    Функция получает на вход таблицу пост user.
    Опционально проводит дропает колонки, проводит OnehotEncoding и CounterEncoding
    а так же может раскодировать возвраст по группам 
    :drop_col:колонки для дропа
    :ohe_col: колонки для OnehotEncoding
    :count_city: и :count_country: (Bool) проводить ли CounterEncoding по колонкам city и country
    :age_group: 'group'/None кодировать ли возвраст
    :norm: (Bool) нормализовать ли данные при CounterEncoding
    """
    user_ = user.copy().drop(drop_col, axis=1) #убираю колонки дропа
    # Новый столбец с категориями возраста
    bins = [0, 18, 30, 60, 120]
    
    if 'age' not in drop_col and (age_group == 'group'):
        user_['age_group'] = pd.cut(user_['age'], bins, labels=['0-17', '18-30', '30-60', '60+'])
        user_ = user_.drop(['age'] , axis =1 )
    elif 'age' not in drop_col and (age_group == 'count'):
        counter_age = CountEncoder(cols=['age'], 
                            return_df=True,
                            normalize=True)
        user_ = counter_age.fit_transform(user_)
    elif 'age' not in drop_col and (age_group == 'none'):
        user_ = user_
    
    #учу CounterEncoder_User
    
    if ('city' not in drop_col) and (count_city == True):
        counter_user = CountEncoder(cols=['city'], 
                            return_df=True,
                            normalize=norm)
        user_ = counter_user.fit_transform(user_)
        
    if ('country' not in drop_col) and (count_country == True):
        counter_country = CountEncoder(cols=['country'], 
                            return_df=True,
                            normalize=norm)
        user_ = counter_country.fit_transform(user_)
        
    
    

    #OHE_User
    for col in ohe_col:
        one_hot = pd.get_dummies(user_[col], prefix=col, drop_first=True)
        user_ = pd.concat((user_.drop(col, axis=1), one_hot), axis=1)
    
    
    return user_

In [7]:
def concat_df(feed_transformd,user_transformd,post_transformd): #feed user post
    """
    Мерджит три таблицы feed, user, post в одну
    """
    df = pd.merge(feed_transformd.sort_values(by ='timestamp' ),
              user_transformd,
              on = 'user_id',
              how = 'left')
    df= pd.merge(df,
             post_transformd,
             on='post_id',
             how ='left')
    return df

In [8]:
def splitter(df):
    """
    Делит сет на трейн и тест 80/20 и выделяет лейблы
    Принимает датаверйм
    Возвращает 
    :X_train,y_train данные для обучения и лейблы
    :X_test,y_test данные для валидации и лейблы
    """
    train = df.iloc[:-int(df.shape[0]*0.2)].copy()
    test = df.iloc[-int(df.shape[0]*0.2):].copy()
    
    train_id = train[['user_id','post_id']]
    test_id = test[['user_id','post_id']] 
    
    #print(f"Значение 0:: на трейне: {(train.groupby(['target'])['post_id'].count())[0]} на тесте: {(test.groupby(['target'])['post_id'].count())[0]} ")
    #print(f"Значение 1:: на трейне: {(train.groupby(['target'])['post_id'].count())[1]} на тесте: {(test.groupby(['target'])['post_id'].count())[1]}\n")
    X_train, X_test = train.drop(['target','timestamp','user_id','post_id','action'], axis =1 ),test.drop(['target','timestamp','user_id','post_id','action'], axis =1 )
    y_train, y_test = train['target'], test['target']
    #print(f'X_train size = {X_train.shape} , X_test size = {X_test.shape} \n y_train size = {y_train.shape} , y_test size = {y_test.shape}')
    
    return [X_train,y_train,X_test,y_test,train_id,test_id]

In [9]:
def get_data_features(df):
    """
    Принимает датафрейм
    Выделяет из колонки timestamp час, месяц и день недели 
    Возвращет датафрейм временными фичами
    """
    df_with_data = df.copy()
    df_with_data['hour'] =  pd.to_datetime(df_with_data['timestamp']).apply(lambda x: x.hour)
    df_with_data['month'] = pd.to_datetime(df_with_data['timestamp']).apply(lambda x: x.month)
    df_with_data['dayofweek'] = pd.to_datetime(df_with_data['timestamp']).apply(lambda x: x.dayofweek)
    return df_with_data

In [10]:
def get_db_with_drop(user__, 
                     post__, 
                     feed__,
                     n_pca,
                     drop_col_,
                     ohe_col_, 
                     ohe_topic = True,
                     count_city = True,
                     count_country = True,
                     age_group_ = True,
                     norm = True):
    """ Функция обедияет в себе вышестоящие функции и принимает необработаные таблицы  feed, user, post.
    Аргументы:
    :drop_col:колонки для дропа
    :ohe_col: колонки для OnehotEncoding
    :count_city: и :count_country: (Bool) проводить ли CounterEncoding по колонкам city и country
    :age_group: 'group'/None кодировать ли возвраст
    :norm: (Bool) нормализовать ли данные при CounterEncoding
    :n_pca: кол-во изерений в новом пространстве, если 0 не проводит pca b tf-idf
    :drop: = True/False дропать ли колонку текст
    :ohe: = True/False проводить ли  OneHotEncoding колонки topic
    :lemma: =True/False проводить ли лемматизацию
    
    Возвращет полностью подготоваленные данные для обаботки:
    :X_train,y_train данные для обучения и лейблы
    :X_test,y_test данные для валидации и лейблы
    """
    
    user_transformd_= transform_user_with_drop_fich(user,drop_col = drop_col_,
                                                    ohe_col=ohe_col_,
                                                    count_city = count_city, 
                                                    count_country = count_country, 
                                                    age_group = age_group_)
    post_transformd_ = post__
    feed_transformd_ = transform_feed(feed__)
    
    df_ = concat_df(feed_transformd_,user_transformd_,post_transformd_)
    #serch_df(df_)
    df_ = get_data_features(df_)
    
    return  splitter(df_)

In [11]:
def feature_imp(model,X_tr): 
    """
    выводит фичи в порядке значимости
    принимает:
    :model: модель catboost
    :X_tr: набот фичей на которых училась модель
    """
    x= {}
    for (feature,col) in zip(model.feature_importances_,X_tr.columns):
        if  feature>0:
            x[col] = feature   
    return dict(sorted(x.items(), key=lambda item: item[1]))

In [12]:
def predict_proba_sort(model,X_ts,y_ts,sort = True, lim = 30):  #model,X_ts,y_ts
    """
    Получает на вход:
    model: обученая модель 
    :X_ts, y_ts:данные тестовой выборки
    :sort:(bool) проводить ли сортировку
    :lim: лимит вывода
    Возвращает датаферм в формате: таргет, предсказание модели, уверенность модели
    """
    y_predict = pd.DataFrame(model.predict(X_ts), columns=['predict'])
    y_predict_proba = pd.DataFrame(model.predict_proba(X_ts), columns=['predict_proba_0','predict_proba_1'] )
    df = pd.merge(X_ts.reset_index(),
             y_ts.reset_index(),
             left_index=True, 
             right_index=True,
             how ='left')
    df = pd.merge(df,
             y_predict,
             left_index=True, 
             right_index=True,
             how ='left')
    df = pd.merge(df,
             y_predict_proba,
             left_index=True, 
             right_index=True,
             how ='left')
    if sort == True:
        df = df.sort_values(by ='predict_proba_1' ,ascending = False)
    
    return df[['target','predict','predict_proba_1']].head(lim)

In [13]:
post = pd.read_csv('post_pca_2_nn.csv')
user = pd.read_csv('user.csv')
feed = pd.read_csv('feed50_50_2mil', parse_dates=["timestamp"]).drop(['index'],axis =1)

In [14]:
#создадим словарь для результатов
dict_result = {}

In [15]:
# обрабатываем данные
drop_col = []
ohe_col=['os','source']#
ohe_topic = True # КОДИРУЕМ ЛИ TOPIC
count_city = True #кодируем ли счетчиком city
count_country = True #кодируем ли country
age_group = 'none' #кодируем ли ваозвраст в группы
norm = True
for pca_num  in [2]: #кол-во PCA в сете
    train_test_list = get_db_with_drop(user, post, feed, 
                                       pca_num ,
                                       drop_col,
                                       ohe_col,
                                       ohe_topic,
                                       count_city,
                                       count_country,
                                       age_group, 
                                       norm) #[X_train,y_train,X_test,y_test]
    X_train , y_train,X_test , y_test = train_test_list[0], train_test_list[1], train_test_list[2], train_test_list[3]
    
    train_id, test_id =  train_test_list[4],train_test_list[5]

In [16]:
#учим модель
#cat boost на трансформированых категориях + PCA 2 на эмбедингай нейросети
    
for i in [2000]:
    
    model_Cat_5 = CatBoostClassifier(random_state = 74) 
    model_Cat_5.set_params(iterations=i,random_state = 74, loss_function='Logloss')
    
    model_Cat_5.fit(
        X_train,
        y_train,
        
        verbose=0,
        plot = 1
        )
    ###eval_set=(X_test, y_test),  
    dict_result[f'Cat_boost_cl_f+ NN={pca_num}, i={i}'] = get_score(y_test,X_test, y_train, X_train, model_Cat_5)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [17]:
dict_result  #0.0832534

{'Cat_boost_cl_f+ NN=2, i=2000': [['precision_tr= 0.9160, recall_tr= 0.9143, f1_tr= 0.9147, ROC-AUC_tr= 0.9143'],
  ['precision= 0.8994, recall= 0.9001, f1= 0.8997, ROC-AUC= 0.9001'],
  ['На трейне предположили 746204, на тесте предположили 214498']]}

In [18]:
#model_Cat_5.save_model('model_Cat_2000it_(2ml)_with_data_nn',
#                         format="cbm")

In [19]:
#Влияние фичей на предсказание в порядке возрастания
feature_imp(model_Cat_5,X_train)

{'topic_covid': 0.004311929370604576,
 'topic_movie': 0.006875304396339395,
 'topic_tech': 0.017680787281081165,
 'topic_entertainment': 0.01960299766589262,
 'topic_politics': 0.04773714260678964,
 'topic_sport': 0.0785893492888314,
 'dayofweek': 0.09551918736757403,
 'PCA_2': 0.11371857196128404,
 'month': 0.15909485554952416,
 'PCA_1': 0.1681133642870348,
 'hour': 0.22300987305683145,
 'gender': 4.441391669238436,
 'os_iOS': 4.904956627036499,
 'source_organic': 4.961367776376124,
 'country': 5.565552446163298,
 'exp_group': 11.678758806121836,
 'age': 27.06136283777829,
 'city': 40.452356474453744}

In [20]:
#достанем предсказания модели для теста
preds = predict_proba_sort(model_Cat_5,X_test, y_test, False, 400000)
preds.head(5)

,target,predict,predict_proba_1
0,1,1,0.736721
1,1,1,0.944665
2,1,1,0.913136
3,1,1,0.996877
4,0,0,0.189634


In [21]:
#склеим предсказания с id пользователя 
preds = pd.merge(test_id.reset_index(),
                   preds,
                   left_index=True, 
                   right_index=True,
                   how ='left').set_index('index')

preds.head(6)

,user_id,post_id,target,predict,predict_proba_1
index,,,,,
1600000,115197,5677,1,1,0.736721
1600001,52361,2040,1,1,0.944665
1600002,28486,4238,1,1,0.913136
1600003,135597,210,1,1,0.996877
1600004,145738,3033,0,0,0.189634
1600005,132135,1504,0,1,0.689930


In [22]:
# посчитаем  hitrate@5 на тестовой выборке
k = 5
hitrate = []

for user in preds['user_id'].unique():
    part = preds[preds['user_id'] == user][:k]
    
    part['compare'] = part['predict'] == part['target']
    a = len(part[part['compare'] == True])
    
    hitrate_k = a / k
    hitrate.append(hitrate_k)
    
print(f"Средний hitrate@5 по пользователям из теста: {round(np.mean(hitrate), 3)}")

Средний hitrate@5 по пользователям из теста: 0.817
